In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import zipfile

import numpy as np
from six.moves import urllib
import tensorflow as tf
from SGD import SGD

In [2]:
def word2vec_basic(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    url = 'http://mattmahoney.net/dc/'

In [3]:
def read_data(filename):
    import pandas as pd
    from tqdm import tqdm
    data = pd.read_excel('bank.xlsx')
    data = data[data['Unnamed: 3'].notnull()]
    tmp_corpus = data['Unnamed: 3'].map(lambda x: x.split(' '))
    
    corpus = []
    for i in tqdm(range(len(tmp_corpus))):
        for line in tmp_corpus[i]:
            corpus += [x for x in line.split()]
    return corpus

In [4]:
vocabulary = read_data('bank.xlsx')
print('Data size', len(vocabulary))

100%|█████████████████████████████████████████████████████████████████████████| 30200/30200 [00:00<00:00, 34424.46it/s]

Data size 192382


In [5]:
#사전을 구축하고 거의 등장하지 않는 단어를 UNK 토큰으로 대체.
vocabulary_size = 500

In [6]:
def build_dataset(words, n_words):
    unique = collections.Counter(words)             # 중복 단어 제거
    orders = unique.most_common(n_words - 1)        # 단어에 대한 빈도 계산. 갯수를 지정하지 않으면 전체 계산.
    count = [['UNK', -1]]
    count.extend(orders)

    # dictionary : (UNK, 0) (the, 1) (of, 2) (and, 3) (one, 4) (in, 5) (a, 6) (to, 7)
    dictionary = {word: i for i, (word, _) in enumerate(count)}

    data = []
    for word in words:
        if word in dictionary:          # word가 dictionary에 존재한다면
            index = dictionary[word]
        else:
            index = 0                   
            count[0][1] += 1            
        data.append(index)

    return data, count, list(dictionary.keys())


data, count, ordered_words = build_dataset(vocabulary, vocabulary_size)
del vocabulary, count               # 사용하지 않는 변수 삭제


In [7]:
# skip-gram 모델에 사용할 학습 데이터를 생성할 함수 작성

def generate_batch(data, batch_size, num_skips, skip_window, data_index):
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window

    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)

    span = 2 * skip_window + 1                      # context = skip_window + target + skip_window
    assert span > num_skips

    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)   # 다음 단어 인덱스로 이동. len(data) = 17005207

    for i in range(batch_size // num_skips):

        targets = list(range(span))     # 1. 0부터 span-1까지의 정수로 채운 다음
        targets.pop(skip_window)        # 2. skip_window번째 삭제
        np.random.shuffle(targets)      # 3. 난수를 사용해서 섞는다.

        start = i * num_skips
        batch[start:start+num_skips] = buffer[skip_window]

        for j in range(num_skips):
            labels[start+j, 0] = buffer[targets[j]]

        # 새로운 요소가 들어가면서 가장 먼저 들어간 데이터 삭제
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)

    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels, data_index

In [8]:
# skip-gram 모델 구축
np.random.seed(1)
tf.set_random_seed(1)

batch_size = 128        # 일반적으로 16 <= batch_size <= 512
embedding_size = 128    # embedding vector 크기
skip_window = 1         # target 양쪽의 단어 갯수
num_skips = 2           # 컨텍스트로부터 생성할 레이블 갯수

valid_size = 16     # 유사성을 평가할 단어 집합 크기
valid_window = 100  # 앞쪽에 있는 분포들만 뽑기 위한 샘플
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # negative 샘플링 갯수

train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

truncated = tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size))
nce_weights = tf.Variable(truncated)
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [9]:
# embeddings 벡터. embed는 바로 아래 있는 tf.nn.nce_loss 함수에서 단 1회 사용
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [10]:
# 배치 데이터에 대해 NCE loss 평균 계산
nce_loss = tf.nn.nce_loss(weights=nce_weights,
                          biases=nce_biases,
                          labels=train_labels,
                          inputs=embed,
                          num_sampled=num_sampled,
                          num_classes=vocabulary_size)
loss = tf.reduce_mean(nce_loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# SGD optimizer
# optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
optimizer = SGD(1.0).minimize(loss)

In [12]:
# 유사도를 계산하기 위한 모델. 학습 모델은 optimizer까지 구축한 걸로 종료.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
# skip-gram 모델 학습
num_steps = 100001

with tf.Session() as session:
    session.run(tf.global_variables_initializer())

    average_loss, data_index = 0, 0
    for step in range(num_steps):
        batch_inputs, batch_labels, data_index = generate_batch(data, batch_size, num_skips, skip_window, data_index)

        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        # 마지막 2000번에 대한 평균 loss 표시
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            print('Average loss at step {} : {}'.format(step, average_loss))
            average_loss = 0

        if step % 10000 == 0:
            sim = similarity.eval()         # (16, 50000)

            for i in range(valid_size):
                valid_word = ordered_words[valid_examples[i]]

                top_k = 8
                nearest = sim[i].argsort()[-top_k - 1:-1][::-1]
                log_str = ', '.join([ordered_words[k] for k in nearest])
                print('Nearest to {}: {}'.format(valid_word, log_str))

    final_embeddings = normalized_embeddings.eval()

Average loss at step 0 : 115.03369140625
Nearest to 중: IBK기업은행,, KB국민은행,, 우리금융, KB증권, 구축, 정상회담, 주요, 中
Nearest to 새: 경쟁, [오전시황], 추진, [파워, 하나금융,, 더, 순익, JB금융,
Nearest to 전일: 수익, 中企, 수준, 하회, 김태오, 지분, 무역협상, 동반
Nearest to 전달: 미·중, 이익, 이어, 복합점포, CIO, 최초, 기업은행, [ET투자뉴스]DGB금융지주,
Nearest to [금융: 금융업, 약보합, 자회사, 돌파, 육성, 시황], 정상회담, 앞둔
Nearest to 대비: 선정, 인상, 개인은, 인사, 전일,, '사자'에, 외인·기관, 대출
Nearest to 실적: DGB금융,, 스타트업, 참여, ..", 매수세에, CEO, 매도, -
Nearest to 기업은행: [포토], 낙폭, 위해, 자산관리, 복합점포, 성공, 블록체인, 소상공인
Nearest to 선정: 대비, 인상, 피해, 신임, [ET투자뉴스]신한지주,, 거래소, 은행권,, 승인
Nearest to 동반: [마감시황], 외, [출발, 금리인상, 상승한, 사외이사, [ET투자뉴스]JB금융지주,, 전일
Nearest to 3분기: 전기,전자, 서비스, 함영주, 윤석헌, 주, KB금융그룹,, 회복, 당기순익
Nearest to 시황]: 혁신, 금융권,, 중기, 코스닥, 美, 반등, [금융, 기록
Nearest to KB국민은행,: 설, 출발, [마켓뷰], 회복, 인터넷은행, 중, [마감]코스피,, 모집
Nearest to 사외이사: 후보, 동반, 수준, 지배구조, 출발, 금리, 국내, 내년
Nearest to 外: [ET투자뉴스]하나금융지주,, 신종자본증권, 위해, 계열사, 매력, 반등, [특징주], 우리은행
Nearest to KB금융: 김기홍, [코스피, 기업銀, 금융위,, [fn, 상승한, 더, 성공
Average loss at step 2000 : 4.4603280

Average loss at step 72000 : 2.7798074108958244
Average loss at step 74000 : 2.814638061761856
Average loss at step 76000 : 2.7501965416073797
Average loss at step 78000 : 2.784823044002056
Average loss at step 80000 : 2.806803152143955
Nearest to 중: 당기순이익, 구축, 마감, 손잡고, 주요, 앞두고, 이후, 최초
Nearest to 새: 선임, 경쟁, [파워, 구축, 추진, 기록.., 하반기, 1위
Nearest to 전일: 코스피], 시각, 시황], 기관은, 전일,, 김태오, 순매도,, 中企
Nearest to 전달: 적정수준, 개최, 이익, 신설, 미·중, CIO, 강보합, 지난해
Nearest to [금융: [이, 금융업, KB금융,, 시각, 시황], 자회사, 코스피,, 육성
Nearest to 대비: 약세, 개인은, 규모, 인상, 인사, 사업, 대출, 하락한
Nearest to 실적: 주요, M&A, DGB금융,, 경쟁, CEO, 다시, 참여, 순익
Nearest to 기업은행: [포토], 당기순익, 복합점포, 이어, 자산관리, 하나금융지주,, 행장, 낙폭
Nearest to 선정: 붕괴, 미중, '사자', 신임, 출시, 오픈, 대비, 승인
Nearest to 동반: 1%, '사자'에, '팔자'에, [출발, 대비, [마감시황], 매도에, 1%대
Nearest to 3분기: 지난해, 2분기, 상반기, 1분기, 전기,전자, 올해, 작년, 이모저모]
Nearest to 시황]: 코스피], 시각, 기록, 전일, 전일,, 지속, 하락폭, 혁신
Nearest to KB국민은행,: [이, KB금융,, [마켓뷰], 설, [ET투자뉴스]하나금융지주,, [시황점검], [ET투자뉴스]신한지주,, 업종
Nearest to 사외이사: 후보, 대표에, 삼성전자, 지배구조, 계열사, 

In [14]:
# embeddings 시각화

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 필요한 패키지와 라이브러리를 가져옴
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
mpl.rcParams['axes.unicode_minus'] = False

plt.rc('font', family='HYMyeongJo-Extra') # For Windows 

import matplotlib.font_manager as fm

def plot_with_labels(low_dim_embs, labels, filename):
    assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords='offset points',
            ha='right',
            va='bottom')
        
    plt.savefig(filename)

try:
    # pylint: disable=g-import-not-at-top
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    
    tsne = TSNE(
        perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = ordered_words[:plot_only]
    plot_with_labels(low_dim_embs, labels, os.path.join('tsne.png'))
    
except ImportError as ex:
    print('Please install sklearn, matplotlib, and scipy to show embeddings.')
    print(ex)